## Imports

In [1]:
from dataloader.load_dt_simulated import load_dt_simulated_dataset, save_dt_simulated_dataset

# Simulated dataset generation

The complete synthetic dataset consists of diffusion weighted data (b-values, gradient orientations, diffusion weighted signals) and the SH expansion coefficients of the fODF which is the target. The simulation starts with sampling random fibre orientations and the corresponding volume fractions for predetermined number of fibre populations. Then given these directions and volume fractions fODF SH coefficients are generated as descibed before. Before the corresponding DW signals can be generated, the eigenvectors of the diffusion tensor need to be generated. Since the eigenvalues are provided as simulation parameters the first eigenvector is chosen to be the fibre population orientation. Then other two random vectors are sampled and by the process of Gram-Schmidt orthonormalization are transformed so that together with the first eigenvector they form an orthonormal basis. These vectors are then used as the eigenvectors of the diffusion tensor model and the corresponding diffusion-weighted signals are produced.

The following cell simulates dataset of 1000 voxels each with 2 fibre populations and maximum SH degree of 8.

In [2]:
dataset = load_dt_simulated_dataset(dataset_size=10, number_of_fibre_populations=2, max_degree=8, vary_gradients=False,seed=1)

[1196.40110327  835.48490032  932.14071758 1427.32379179 1110.24972727
 1257.12394019 1332.90361032  914.71885078 1780.49373505  931.60727773
  713.58362313 1236.37627521  854.97620799  770.57996446  622.8480159
 2417.54229437 1521.86597698  722.64805041  831.62000715  632.67005804
 2062.80984427  564.58045949 1916.39228475  760.6752752   847.41189081
 1922.84934019 1740.96731171  903.1455002  1929.3293631  1253.79614255
 1309.39126014 1545.91841256 2067.43998801 1086.45907822 1706.08734107
 1586.06309896  990.10056455 1719.345279   2293.21236665  882.00583134
 1406.46255342  863.8360829   597.84538409 2064.012558    638.81595984
 1459.97487998 1181.80563658  720.72755606  888.08578839 2394.46325972
 1403.69355482 2353.16695945 1254.57008594  659.48635204 1547.45066721
  920.64624848 1058.28883811 1072.92152934  561.09857238 1418.32810377
 2373.60732409 1529.21909941 1339.73811799 1611.56093342  685.66950116
  736.69889409 1225.62653824  588.64962007 1391.5143811   701.682714
  851.458

The following three sections show synthetic data for one voxel.

## SH coefficients of fODF

In [3]:
dataset[0][0]

array([ 0.28209479, -0.23487825,  0.02369006, -0.28264077, -0.12781006,
       -0.05303198,  0.25231634, -0.04595966,  0.2355048 , -0.05665353,
        0.21512753,  0.20747266,  0.05332506,  0.19581961, -0.5264566 ,
        0.24650176, -0.16607579, -0.12970463,  0.02508347, -0.25557612,
        0.10639847, -0.12574753, -0.24675288, -0.05801727, -0.26728264,
        0.27111804,  0.19828596,  0.19398731, -0.49612805,  0.17504913,
       -0.23548985,  0.23897467,  0.05143837,  0.03179002,  0.25278147,
       -0.16965795,  0.03516818,  0.24415041,  0.05749774,  0.2989179 ,
       -0.10815761, -0.19397131, -0.18638991, -0.24944553,  0.39882963])

## b-vals, gradient orientations, signals

In [4]:
dataset[1][0]

(array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000]),
 array([[ 0.36353657, -0.79057113, -0.61636162,  0.47329009, -0.82269029,
          0.89697745, -0.94988454, -0.16577181, -0.81933594, -0.8108305 ,
          0.87454161,  0.91075499, -0.29634718,  0.07737383, -0.0756303 ,
          0.18488546,  0.53113688, -0.67804166,  0.52939376, -0.0155802 ,
          0.0529998 , -0.9090384 ,  0.12865447,  0.19786134, -0.71456792,
          0.5073

## Fibre orientations

In [5]:
dataset[2][0]

array([[ 0.61898402, -0.77509971,  0.1268039 ],
       [-0.74957682, -0.59759348, -0.28463418]])

## Saving the datasets

In [6]:
dataset1 = load_dt_simulated_dataset(dataset_size=100000, number_of_fibre_populations=1, max_degree=8, vary_gradients=False, seed=1)
save_dt_simulated_dataset(dataset1, './data/dt_simulated_1_populations')

In [7]:
dataset2 = load_dt_simulated_dataset(dataset_size=100000, number_of_fibre_populations=2, max_degree=8, vary_gradients=False, seed=1)
save_dt_simulated_dataset(dataset2, './data/dt_simulated_2_populations')

In [8]:
dataset3 = load_dt_simulated_dataset(dataset_size=100000, number_of_fibre_populations=3, max_degree=8, vary_gradients=False, seed=1)
save_dt_simulated_dataset(dataset1, './data/dt_simulated_3_populations')